# [IMPORTANT!] EXPERIMENTAL


Text2SQL V1 is an experimental solution that combines calls to Vertex LLM with BQ executions. It is important to note that executing the generated SQL query could potentially modify the data and shouldn't be run on production datasets.

Experiments are focused on validating a prototype and are not guaranteed to be released. Experiments are covered by the Pre-GA Offerings Terms of the Google Cloud Platform Terms of Service. They are not intended for production use or covered by any SLA, support obligation, or deprecation policy and might be subject to backward-incompatible changes.

The underlying model/API is GA with billing enabled.


# Generative AI: Text2SQL V1 Instructions [External]

---
This notebook demonstrates the Experimental Text2SQL V1 capabilities offering on Vertex AI.

Please note that Text2SQL V1 is in the early stages of development. The GCP project does not require allowlisting.

---
This notebook is composed of 2 parts.
1. Deployment of Vertex AI Model and Vertex AI Endpoint on the user project
2. Text2SQL using the endpoint calls to the deployed endpoint

Requirements:
- google-cloud-aiplatform>=1.24.1

In [ ]:
# !pip install google-cloud-aiplatform --upgrade

In [2]:
# @title # Authentication. { display-mode: "form"}
from google.cloud import aiplatform
from google.cloud import bigquery

In [ ]:
# @title # Deploying Endpoint to Project
# @markdown We provide a custom container **us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/text2sql_bq** which can be deployed to your project directly. This is a one time action as long as there is a deployed endpoint.

# @markdown Specify the following parameters for the endpoint deployment
MODEL_DISPLAY_NAME = "text2sql-model"
IMAGE_URI = "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/text2sql_bq"
LOCATION = "us-central1"

PROJECT_ID="wmt-7fbls2a91f025anb93e025b02g" # @param {type:"string"}
MACHINE_TYPE = "e2-standard-2" # @param {type:"string"}
# @markdown Furthermore, we need a service account that the DeployedModel's container runs as. Please give "BigQuery Data Viewer", "Vertex AI Service Agent" and "Vertex AI User" permissions in IAM for the service account used to run the Text2SQL pipeline

# @markdown   Example: text2sqldemo@your-project.iam.gserviceaccount.com. You can refer [here](https://cloud.google.com/iam/docs/service-accounts-create) for more information.
SERVICE_ACCOUNT = "svc-deploy-mgmt@wmt-7fbls2a91f025anb93e025b02g.iam.gserviceaccount.com" # @param {type:"string"}


def upload_model():
  model = aiplatform.Model.upload(
      display_name=MODEL_DISPLAY_NAME,
      serving_container_image_uri=IMAGE_URI,
  )
  return model

def deploy_model_to_endpoint(deployed_model):
  endpoint = deployed_model.deploy(
    machine_type=MACHINE_TYPE, service_account=SERVICE_ACCOUNT
  )
  return endpoint

def undeploy_model_and_delete_endpoint(endpoint_id):
  endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}")

  endpoint.undeploy_all()
  endpoint.delete()

aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Endpoint deployment
model = upload_model()
endpoint = deploy_model_to_endpoint(model)
print(f'Endpoint id is: {endpoint.name}')

Creating Model
Create Model backing LRO: projects/896267025569/locations/us-central1/models/2454538762730864640/operations/1235255890232213504
Model created. Resource name: projects/896267025569/locations/us-central1/models/2454538762730864640@1
To use this Model in another session:
model = aiplatform.Model('projects/896267025569/locations/us-central1/models/2454538762730864640@1')
Creating Endpoint
Create Endpoint backing LRO: projects/896267025569/locations/us-central1/endpoints/5400240364705546240/operations/7227998881890435072


In [ ]:
# @title # Running Text2SQL. { display-mode: "form" }

# Variables added if a user has already deployed the endpoint
LOCATION = "us-central1"
PROJECT_ID="wmt-7fbls2a91f025anb93e025b02g" # @param {type:"string"}

# @markdown Once you have deployed the endpoint (check previous section) with the Text2SQL server, specify the following parameters. For more information, check the [endpoint documentation](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints). Endpoint deployment might take a few minutes to run.
ENDPOINT_ID = "5400240364705546240" #@param {type: "string"}

# @markdown Now we will build the Text2SQL endpoint request which will called to convert the natural language english question into a BQ SQL query.
# @markdown Questions should not contain any instructions related to modifying the table such as "delete", "change", "drop", "remove", "update" and only contain language related to querying the data such as "What" and "how many".
NL_QUESTION = "Get total number of people from state of WA grouped by name and state, limit results by 10 and sort by highest to lowest." # @param {type:"string"}
BQ_DATASET = "bigquery-public-data.usa_names" # @param {type:"string"}

# @markdown ## Important!
# @markdown When the BQ dataset is too large, the underlying prompt might be larger than the supported token limit of the LLM, therefore we provide a way to provide filters that limit tables/columns from the schema that the model will have access to.
# @markdown For more information on LLM token limits, check [here](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#foundation_models).

# @markdown If the Text2SQL endpoint requests fails with an error similar to `[ORIGINAL ERROR] generic::invalid_argument: The model supports up to 8192 input tokens, but received 2100001 tokens.`, specify the parameters below.

# @markdown The following are optional parameters but help mitigate the issue with the token limit:

# @markdown 1. List of strings of BQ table names denoting subset from the BQ dataset that should be considered. Use the full table name, e.g. **["bigquery-public-data.london_bicycles.cycle_stations", ...]**.
BQ_TABLES_FILTER = [] #@param {type:"raw"}
# @markdown 2. Optional dict of { table name strings : list of columns names } denoting columns to be considered for each table. The model will only see the table bigquery-public-data.london_bicycles.cycle_stations and think that it only has 3 columns:  "id", "name", "install_date". Should only be used when execution_mode="manual". E.g. **{ "bigquery-public-data.london_bicycles.cycle_stations": ["id", "name", "install_date"] }**
BQ_COLUMNS_FILTER = {} #@param {type:"raw"}

In [ ]:
# @title ### Helper: building the request

# @markdown We can control the number of samples made to the LLM which improves the chance of the generated sql query to be correct.
# @markdown A lower value translated to lower latency but worse performance. Additionaly, the request cost will be proportional to `NB_SAMPLES`.
NB_SAMPLES = 5 # @param {type:"integer"}
TEMPERATURE = 0.8 # @param {type:"number"}

def build_request(bq_tables_filter = None, bq_columns_filter = None):
  """Creates a single endpoint request."""

  example_input = [
    {
      "text": NL_QUESTION,
      "bq_dataset": BQ_DATASET,
      "project_id": PROJECT_ID,
    }]
  if bq_tables_filter:
    example_input[0]["bq_tables_filter"] = bq_tables_filter
  if bq_columns_filter:
    example_input[0]["bq_columns_filter"] = bq_columns_filter

  params = {
    "temperature":0.8,
    "nb_samples":NB_SAMPLES,
  }

  return example_input, params

In [ ]:
# @title ### Call Text2SQL endpoint
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# build request
example_input, params = build_request(bq_columns_filter=BQ_COLUMNS_FILTER, bq_tables_filter=BQ_TABLES_FILTER)

# Query via Endpoint directly
endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}")

# The endpoint request
response = endpoint.predict(instances=example_input, parameters=params)

# response.predictions[0] is a dictionary:
# {
#  "status" "result" or "exception",
#  "sql_query" : "Generated SQL query string"
# }
response.predictions[0]

In [ ]:
sql_query = response.predictions[0]['sql_query']
print("Generated SQL query:", sql_query)

In [ ]:
#@title [Optional] Execute generated SQL query on BQ data
client = bigquery.Client(project=PROJECT_ID)
df_res = client.query(sql_query).to_dataframe()

df_res

In [ ]:
#@title [Optional] Cleaning up

# @markdown When you are done, undeploy and delete the endpoint. Uncomment the following code.
# undeploy_model_and_delete_endpoint(ENDPOINT_ID)